# Ground State Energy Estimation for Photosynthesis with Quantum Circuits

Artificial photosynthesis mimics natural photosynthesis by
absorbing solar light and splitting water into O2, protons (H+) and electrons (e).

The electrons extracted from water can reduce protons or CO2
to produce energy carrier fuels such as H2 or hydrocarbons

Water oxidation reaction
2H2O -> O2 + 4(H+) + 4e

Proton reduciton reaction:
2(H+) + 2e -> H2

CO2 reduciton reactions:
- CO2 + 2(H+) + 2e -> CO + H2O    (CO2 Reduction 1)

- CO2 + 6(H+) + 6e -> CH3OH + H2O (CO2 Reduction 2)

- CO2 + 8(H+) + 8e -> CH4 + 2H2O  (CO2 Reduction 3)

Typical examples of water oxidation:
Co_4O_4 catalysis (https://pubs.acs.org/doi/10.1021/ja202320q)

In [ ]:

import os
import re
import sys
import time
import cirq
import numpy as np
# from pyLIQTR.GSE.GSE import GSE
from openfermionpyscf import run_pyscf
from openfermion.chem import MolecularData
from pyLIQTR.utils.qsp_helpers import circuit_decompose_once
from pyLIQTR.gate_decomp.cirq_transforms import clifford_plus_t_direct_transform
from pyLIQTR.utils.utils import count_T_gates
from pyLIQTR.PhaseEstimation.pe import PhaseEstimation

In [5]:
def extract_number(string):
    number = re.findall(r'\d+', string)
    return int(number[0]) if number else None

def count_gates(cpt_circuit):
    count = 0
    for moment in cpt_circuit:
        count += len(moment)
    return count

def num_fermion_qubits(ham):
    n_qubits = 0
    for term in ham.terms:
        for op in term:
            qubit = op[0]
            if qubit >= n_qubits:
                n_qubits = qubit+1
    return n_qubits

In [6]:
def estimate_gse(circuit, outdir, circuit_name='gse_circuit', write_circuits=False):
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    
    subcircuit_counts = dict()
    t_counts = dict()
    clifford_counts = dict()
    gate_counts = dict()
    subcircuit_depths = dict()
    
    outfile_data = f'{outdir}{circuit_name}_high_level.dat'

    for moment in circuit:
        for operation in moment:
            gate_type = type(operation.gate)
            if gate_type in subcircuit_counts:
                subcircuit_counts[gate_type] += 1
            else:
                decomposed_circuit = circuit_decompose_once(circuit_decompose_once(cirq.Cirquit(operation)))
                cpt_circuit = clifford_plus_t_direct_transform(decomposed_circuit)
                
                if write_circuits:
                    outfile_qasm_decomposed = f'{outdir}{str(gate_type)[8:-2]}.decomposed.qasm'
                    outfile_qasm_cpt = f'{outdir}{str(gate_type)[8:-2]}.cpt.qasm'
                    outfile_qasm_cpt = f'{outdir}{str(gate_type)[8:-2]}.cpt.qasm'
                    cirq.QasmOutput(decomposed_circuit).save(outfile_qasm_decomposed)
                    cirq.QasmOutput(cpt_circuit).save(outfile_qasm_cpt)
                
                subcircuit_counts[gate_type] = 1
                subcircuit_depths[gate_type] = len(cpt_circuit)
                t_counts[gate_type] = count_T_gates(cpt_circuit)
                gate_counts[gate_type] = count_gates(cpt_circuit)
                clifford_counts[gate_type] = gate_counts[gate_type] - t_counts[gate_type]
                
                
    total_gate_count = 0
    total_gate_depth = 0
    total_T_count = 0
    total_clifford_count = 0
    for gate in subcircuit_counts:
        total_gate_count += subcircuit_counts[gate] * gate_counts[gate]
        total_gate_depth += subcircuit_counts[gate] * subcircuit_depths[gate]
        total_T_count += subcircuit_counts[gate] * t_counts[gate]
        total_clifford_count += subcircuit_counts[gate] * clifford_counts[gate]
    with open(outfile_data, 'w') as f:
        total_gate_count 
        f.write(f'Logical Qubit Count: {len(circuit.all_qubits())}\n')
        f.write(f'Total Gate Count: {total_gate_count}\n')
        f.write(f'Total Gate Depth: {total_gate_depth}\n')
        f.write(f'Total T Count: {total_T_count}\n')
        f.write(f'Total Clifford Count: {total_clifford_count}\n')
        f.write('Subcircuit Info:\n')
        for gate in subcircuit_counts:
            f.write(f'{gate}\n')
            f.write(f'Subcircuit Occurrences: {subcircuit_counts[gate]}\n')
            f.write(f'Gate count: {gate_counts[gate]}\n')
            f.write(f'Gate depth: {subcircuit_counts[gate]}\n')
            f.write(f'T Count: {t_counts[gate]}\n')
            f.write(f'Clifford Count: {clifford_counts[gate]}\n')

In [7]:
def load_pathway_xyz(fname, pathway=None):
    coordinates_pathway = []
    with open(fname) as f:
        data = f.readlines()
        coordinates_pathway = []
        for k, line in enumerate(data):
            if 'multiplicity' in line or 'charge' in line:
                coords_list = []
                geo_name = None
                if len(line.split(',')) > 2:
                    geo_name = line.split(',')[2]

                # Use a regular expression to extract the multiplicity value
                match = re.search(r"multiplicity\s*=\s*(\d+)", line)
                if match:
                    multiplicity = int(match.group(1))  # Convert the string to an integer
                match = re.search(r"charge\s*=\s*(\d+)", line)
                if match:
                    charge = int(match.group(1))  # Convert the string to an integer

                nat = int(data[k-1].split()[0])
                print(f'{geo_name} Multiplicity: {multiplicity}  total no. of atoms={nat}, charge = {charge}')
                coords_list.append([nat, charge, multiplicity])
                for i in range(nat):
                    tmp = data[k+1+i].split()
                    aty = tmp[0]
                    xyz = [float(tmp[i]) for i in range(1,4)]
                    coords_list.append([aty, xyz])

                if geo_name is not None and pathway is not None:
                    order = extract_number(geo_name)
                    if order is not None and order in pathway:
                        coordinates_pathway.append(coords_list)
                else:
                    coordinates_pathway.append(coords_list)
    return coordinates_pathway

In [10]:
molecular_hamiltonians = []
pathway0 = [5, 10, 28, 29, 30, 31, 32, 33]
pathway1 = [2, 1, 14, 15, 16, 17, 18, 19]
pathway2 = [3, 1, 14, 15, 16, 20, 21, 22, 23]
pathway3 = [27, 1, 14, 15, 16, 24, 25, 26]
# Water oxidation via Co4O4 catalyst.
# water oxidation
coordinates_pathway = load_pathway_xyz('../data/water_oxidation_Co4O4.xyz', pathway=pathway0)

# co2 reduction
coordinates_pathway = load_pathway_xyz('../data/CO2_reduciton_CoPc.xyz')

# Set calculation parameters.
run_scf = 1
run_mp2 = 0
run_cisd = 0
run_ccsd = 0
run_fci = 0

# Set molecule parameters.
basis = 'sto-3g'
multiplicity = 1
n_points = 40
bond_length_interval = 3.0 / n_points
active_space_frac = 5 # 1 over n

 D1
 Multiplicity: 1  total no. of atoms=26, charge = 0
 D2
 Multiplicity: 1  total no. of atoms=26, charge = 0
 D3
 Multiplicity: 1  total no. of atoms=26, charge = 0
 D4
 Multiplicity: 1  total no. of atoms=26, charge = 0
 D5
 Multiplicity: 1  total no. of atoms=26, charge = 0
 D6
 Multiplicity: 1  total no. of atoms=26, charge = 0
 D7
 Multiplicity: 1  total no. of atoms=26, charge = 0
 D8
 Multiplicity: 1  total no. of atoms=26, charge = 0
 D9
 Multiplicity: 1  total no. of atoms=26, charge = 0
 D10
 Multiplicity: 1  total no. of atoms=26, charge = 0
 D11
 Multiplicity: 1  total no. of atoms=26, charge = 0
 D12
 Multiplicity: 1  total no. of atoms=26, charge = 0
 D13
 Multiplicity: 1  total no. of atoms=26, charge = 0
 D14
 Multiplicity: 2  total no. of atoms=25, charge = 0
 D15
 Multiplicity: 3  total no. of atoms=24, charge = 0
 D16
 Multiplicity: 2  total no. of atoms=23, charge = 0
 D16
 Multiplicity: 4  total no. of atoms=23, charge = 0
 D17
 Multiplicity: 2  total no. of atom

In [11]:
print('\nGenerating the electronic Hamiltonain along the reaction pathway!\n')

if len(coordinates_pathway) > 0:
    # generate the Hamiltonians
    for coords in coordinates_pathway:
        nat, charge, multi = [int(coords[0][j]) for j in range(3)]
        print(f'\nGenerating the qubit Hamiltonian for the molecule: \n {coords}\n')
        #print(f"\nNatoms={nat} charge={charge} multiplicity={multi}")

        # set molecular geometry in pyscf format
        basis = 'cc-pvdz'
        #basis = "STO3G"
        geometry = []
        for k, coord in enumerate(coords[1:]):
            coord_str = ' '.join(map(str, coord[1]))
            if k == nat-1:
                coord_str = f'{coord[0]} {coord_str}'
            else:
                coord_str = f'{coord[0]} {coord_str};\n'
            atom = (coord[0], tuple(coord[1]))
            geometry.append(atom)

        molecule = MolecularData(geometry, basis, multi, charge=charge, description="catalyst")

        print(f'no, of atoms = {len(geometry)}')
        # Run pyscf.
        molecule = run_pyscf(molecule,
                             run_scf=run_scf,
                             run_mp2=run_mp2,
                             run_cisd=run_cisd,
                             run_ccsd=run_ccsd,
                             run_fci=run_fci)

        print(f'number of orbitals          = {molecule.n_orbitals}')
        print(f'number of electrons         = {molecule.n_electrons}')

        print(f'number of qubits            = {molecule.n_qubits}')
        print(f'Hartree-Fock energy         = {molecule.hf_energy}')
        nocc = molecule.n_electrons // 2
        nvir = molecule.n_orbitals - nocc
        sys.stdout.flush()

        # get molecular Hamiltonian
        active_space_start =  nocc - nocc // active_space_frac # start index of active space
        active_space_stop = nocc + nvir // active_space_frac   # end index of active space

        print(f'active_space start = {active_space_start}')
        print(f'active_space stop  = {active_space_stop}')
        sys.stdout.flush()
        sys.exit()

        molecular_hamiltonian = molecule.get_molecular_hamiltonian(
        occupied_indices=range(active_space_start),
        active_indices=range(active_space_start, active_space_stop)
        )

        # shifted by HF energy
        molecular_hamiltonian -= molecule.hf_energy
        molecular_hamiltonians.append(molecular_hamiltonian)


 Generating the electronic Hamiltonain along the reaction pathway!


Generating the qubit Hamiltonian for the molecule: 
 [[61, 1, 2], ['C', [4.12130305099727, 1.964084137e-05, 1.920115422e-05]], ['O', [4.11197252510114, -1.17186167143454, -0.00182722827519]], ['O', [4.11197225571057, 1.17190094534303, 0.00183295995384]], ['H', [1.40328541604672, -1.22801072e-06, -7.906539e-07]], ['Co', [-0.02248508039715, -8.2615969e-07, -5.1439594e-07]], ['N', [-0.07553930342398, 0.00239993718526, -1.9090365390745]], ['N', [-0.07190821239349, -1.90921681053614, -0.00240164634891]], ['N', [-0.05642464843702, 2.38848644152891, -2.3823776279747]], ['N', [-0.05642934748584, -2.38247835550713, -2.38838779784081]], ['C', [-0.06883061802419, 1.11619968304282, -2.73963511282038]], ['C', [-0.06883151881768, -1.10929660546526, -2.74244251141559]], ['C', [-0.05611125720557, -2.7397940830474, -1.11587153206023]], ['C', [-0.05610728678487, -2.74260056404773, 1.1089647931155]], ['C', [-0.08108470757371, 0.7076681

In [ ]:
E_min = -4000
E_max = -3000
omega = E_max - E_min
t = 2*np.pi/omega
phase_offset = E_max * t

for i in molecular_hamiltonians:
    molecular_hamiltonian = molecular_hamiltonians[i]
    trotter_order = 2
    trotter_steps = 1
    n_qubits = molecular_hamiltonian.n_qubits
    # note that we would actually like within chemical precision
    # which should take > 10 bits of precision, it just takes a 
    # really long time to run so a scaling argument will be needed
    bits_precision = 1
    
    gse_args = {
        'trotterize' : True,
        'mol_ham'    : molecular_hamiltonian,
        'ev_time'    : 1,
        'trot_ord'   : trotter_order,
        'trot_num'   : trotter_steps
    }

    
    init_state = [0] * n_qubits
    
    print('starting')
    t0 = time.perf_counter()
    gse_inst = PhaseEstimation(
        precision_order=bits_precision,
        init_state=init_state,
        phase_offset=phase_offset,
        include_classical_bits=False,
        kwargs=gse_args
    )
    gse_inst.generate_circuit()
    t1 = time.perf_counter()
    print(f'Co4O4 time to generate high level number {i} : {t1 - t0}')
    gse_circuit = gse_inst.pe_inst.pe_circuit
    
    print('Estimating Co4O4 circuit {i}')
    t0 = time.perf_counter()
    estimate_gse(gse_circuit, outdir='GSE/', circuit_name=f'Co4O4_{i}')
    t1 = time.perf_counter()
    print(f'Time to estimate Co4O4: {t1-t0}')